In [1]:
from PIL import ImageFont, ImageDraw, Image
import numpy as np
import cv2
import mediapipe as mp
# 한글을 출력하기 위한 함수
def draw_text(img, text, position, font_size, font_color):
    font_path = "C:/Windows/Fonts/gulim.ttc"  # Windows에서 Gulim 폰트 경로

    # opencv 이미지를 PIL이미지로 변
    
    img_pil = Image.fromarray(img)

    # PIL Draw 객체 생성
    draw = ImageDraw.Draw(img_pil)

    # 폰트 스타일 지정
    font = ImageFont.truetype(font_path, font_size)

    # PIL 이미지에 텍스트 입력
    draw.text(position, text, font=font, fill=font_color)
    return np.array(img_pil) # 최종 numpy array 로 이미지 형태 반환

In [5]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# 손가락의 개폐 상태를 기반으로 '가위', '바위', '보' 포즈를 판별하는 함수
def get_hand_pose(hand_landmarks):
    # 손가락 끝과 PIP(손가락 중간 관절) 사이의 거리를 비교하여 손가락이 펴져 있는지 확인
    open_fingers = []
    for i in [4, 8, 12, 16, 20]:  # 엄지부터 새끼손가락까지의 끝 랜드마크 인덱스
        tip = hand_landmarks.landmark[i]  # 손가락 끝
        pip = hand_landmarks.landmark[i - 2]  # 손가락 중간 관절

        # if handedness.classification[0].label == "Right":  # 오른손인 경우 판별하기

        # 엄지는 x좌표를, 나머지 손가락은 y좌표를 사용하여 개폐 상태 확인 - 오른손 기준
        if i == 4:  # 엄지손가락인 경우
            open_fingers.append(tip.x > pip.x)
        else:
            open_fingers.append(tip.y < pip.y)
    
    # '가위', '바위', '보' 판별
    if open_fingers.count(True) == 5:
        return "놓기"
    elif open_fingers.count(True) == 0:
        return "잡기"
    else:
        return "모름"

cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, image = cap.read()
    if not success:
        continue

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            # 손 랜드마크 그리기
            mp_drawing.draw_landmarks(
                image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2))
            
            # 포즈 판별 및 출력
            pose = get_hand_pose(hand_landmarks)
            
            # 한글 출력을 위한 작업
            image = draw_text(image, pose, (10, 50), 30, (255, 255, 255))

    cv2.imshow('Hand Pose', image)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()